
# IMPORTANDO ARQUIVOS BIBTEX e EXPORTANDO EM FORMATAÇÃO YAML
- Arquivo texto em formato BIBTEX
- Objetivo: Data quality de arquivos BIBtex
- Extensão: .bib
- Biblioteca: pybtex

In [381]:

from pybtex.database.input import bibtex
from pybtex.database import BibliographyData, Entry
import os

# #################################################################
## Inicia importanção do arquivo
# #################################################################

In [382]:
def arquivos(path, fendwith = ''):
    if fendwith not in '':
        caminhos = {nome : os.path.join(path, nome) for nome in os.listdir(path) if nome.endswith(fendwith)}
    else:
        caminhos = {nome : os.path.join(path, nome) for nome in os.listdir(path)}
    return caminhos

def author_names(author):
    try:
        return author.persons['author'][0:]
    except:
        return {'author': [(u'none, none')]}

def join_names(person):
    try:
        return person.last_names[0] + ', ' + person.first_names[0]
    except:
        return []

def type_output_file(f_type, filename, path):
    
    import yaml
    import json
    import csv

    path_complete = path + filename + '.' + f_type

    if f_type == 'yaml':
        with open(path_complete, 'w') as nfile:
            for data in dict.items():
                yaml.dump(data, nfile)  # insere os dados na configuração YAML

    if f_type == 'json':
        with open(path_complete, 'w') as nfile:
            for data in dict.items():
                json.dump(data, nfile)  # insere os dados na configuração YAML

    if f_type == 'csv':
        with open(path_complete, 'w', -1, "utf-8") as nfile:
        #head
            nfile.write('{0}\n'.format('§ '.join(str(x) for x in list_file_fields)))
            #row
            for row in dict.keys():
                # print(dict[row])
                nfile.write('{0}\n'.format('§ '.join(str(x) for x in dict[row].values())))

## Inicia tratamento dos dados BIBTEX


ref bibtex doc: http://paginapessoal.utfpr.edu.br/jamhour/publicacoes/arquivos/00_Compilado_JabRef_dez2015.pdf





In [383]:
source_path = "C:\\Users\\victo\\PycharmProjects\\BibtexDataQuality\\source\\"

lst_files = arquivos(source_path, '.bib')
parser = bibtex.Parser()
# file = parser.parse_file(lst_files['ACM.bib'])
dict_file_fields = {}
list_file_fields = []

#verifica a estrutura de campos para cada tipo de publicação
#cria um dicionario usando como chave a fonte e o tipo de publicação
for f in lst_files:
    parser = bibtex.Parser()
    file = parser.parse_file(lst_files[f])
    f_name = f.replace('.bib','')
    for i in file.entries.values():
        
        for a in sorted(list(i.fields.keys())):
           
            if a not in list_file_fields:
                list_file_fields.append(a)
list_file_fields.remove('ISSN')

In [387]:
lst_files = arquivos(source_path)
parser = bibtex.Parser()
# file = parser.parse_file(lst_files['IEEE.bib'])
dict = {}

for f in lst_files:
    
    ############# IEEE
    if 'IEEE' in f :
        parser = bibtex.Parser()
        file = parser.parse_file(lst_files[f])

        for i in file.entries.values():
            key = i.key
            dict[key] = {i.fields.get('fields', fields) : i.fields.get(fields, '')\
                        for fields in list_file_fields}
            
            dict[key]['type_publication'] = i.type
            dict[key]['author'] = [join_names(person) for person in i.persons.get('author', '')]

            dict[key].update({'keywords' : dict[key]['keywords'].split(';')})
    
    ############## ACM
    if 'acm' in f:
        parser = bibtex.Parser()
        file = parser.parse_file(lst_files[f])

        for i in file.entries.values():
            key = i.key
            dict[key] = {i.fields.get('fields', fields) : i.fields.get(fields, '')\
                        for fields in list_file_fields}
            
            dict[key]['type_publication'] = i.type
            dict[key]['author'] = [join_names(person) for person in i.persons.get('author', '')]

            dict[key].update({'keywords' : dict[key].get('keywords', '').split(';')})

            if i.type == 'inbook':
                dict[key].update({'doi' : dict[key].get('url', '').replace('https://doi.org/', '')})

    ############## Science Direct
    if 'ScienceDirect' in f:
        parser = bibtex.Parser()
        file = parser.parse_file(lst_files[f])

        for i in file.entries.values():
            key = i.key
            dict[key] = {i.fields.get('fields', fields) : i.fields.get(fields, '')\
                        for fields in list_file_fields}
            
            dict[key]['type_publication'] = i.type
            dict[key]['author'] = [join_names(person) for person in i.persons.get('author', '')]
            dict[key].update({'keywords' : dict[key]['keywords'].split(';')})
            dict[key].update({'doi' : i.fields['doi'].replace('https://doi.org/', '')})


In [388]:
print(dict['10.1145/3502771.3502781'])
# print({key : dict['10.1145/3411764.3445130'][key] for key in ['author', 'title', 'keywords', 'year', 'type_publication', 'doi']})

{'abstract': 'Cyber-physical systems (CPS) have been developed in many industrial sectors and application domains in which the quality requirements of data acquired are a common factor. Data quality in CPS can deteriorate because of several factors such as sensor faults and failures due to operating in harsh and uncertain environments. How can software engineering and artificial intelligence (AI) help manage and tame data quality issues in CPS? This is the question we aimed to investigate in the SEA4DQ workshop. Emerging trends in software engineering need to take data quality management seriously as CPS are increasingly datacentric in their approach to acquiring and processing data along the edge-fog-cloud continuum. This workshop provided researchers and practitioners a forum for exchanging ideas, experiences, understanding of the problems, visions for the future, and promising solutions to the problems in data quality in CPS. Examples of topics include software/hardware architecture

## Output

In [ ]:

output_path = "C:\\Users\\victo\\PycharmProjects\\BibtexDataQuality\\output\\"

type_output_file('csv', 'output', output_path)

## Inicia tratamento dos dados CSV

In [ ]:
# print(lst)

# #################################################################
# #Verificar se os dados estão corretos em cada coluna
# #################################################################

# lst_field = ['chave', 'author', 'title', 'keywords', 'abstract', 'year', 'type_publication', 'doi']
# for f in lst_field:
#     for q in lst[0:]:
#         print(q[f])